In [336]:
import numpy as np; import pandas as pd
from typing import Literal, Tuple
import random as rd
from scipy.optimize import linprog
import Toolbox_20250801 as toolbox

In [337]:
A = np.matrix(np.random.randint(-100, 100, (4, 3)))
maximinValue_row = toolbox.maximin(A, "row")[0]
worstValue_row = A.min()
print(A, maximinValue_row, worstValue_row, sep = '\n')

[[ 16  17 -31]
 [-45 -13  22]
 [ 41  18  22]
 [-73  79  66]]
26.017142857142844
-73


In [338]:
B = np.matrix(np.random.randint(-100, 100, (4, 3)))
maximinValue_col, maximinStrategy_col = toolbox.maximin(B, "column")
print(B, maximinValue_col, maximinStrategy_col.T, sep = '\n')

[[-41  15  83]
 [-19  55 -43]
 [-47  67 -25]
 [ 92  54  57]]
31.38554216867469
[[0.         0.75903614 0.24096386]]


In [339]:
riskAversion_row = rd.random()
minimumRequiredValue_row = riskAversion_row*maximinValue_row + (1-riskAversion_row)*worstValue_row
print(riskAversion_row, minimumRequiredValue_row)

0.10346885063925715 -62.7548100349883


In [340]:
strategySpace_row = toolbox.thresholdVertices(A, minimumRequiredValue_row, "row")
print(strategySpace_row)

               Vertex_0  Vertex_1  Vertex_2  Vertex_3  Vertex_4  Vertex_5
Probability_0       0.0       0.0       1.0  0.115114    0.0000   0.00000
Probability_1       0.0       1.0       0.0  0.000000    0.3659   0.00000
Probability_2       1.0       0.0       0.0  0.000000    0.0000   0.08987
Probability_3       0.0       0.0       0.0  0.884886    0.6341   0.91013


In [341]:
P = np.matrix(strategySpace_row.to_numpy())
print(P)

[[0.         0.         1.         0.11511449 0.         0.        ]
 [0.         1.         0.         0.         0.36589964 0.        ]
 [1.         0.         0.         0.         0.         0.08987009]
 [0.         0.         0.         0.88488551 0.63410036 0.91012991]]


In [342]:
enhancedPayoff_col, enhancedStrategy_col = toolbox.maximin(P.T*B, player = "column")
print(enhancedPayoff_col, enhancedStrategy_col.T)
print(maximinValue_col, maximinStrategy_col.T)
# Even with the same strategy, the actual payoff may be enhanced, 
# due to the opponent is limited from playing the worst pure strategy

31.38554216867469 [[0.         0.75903614 0.24096386]]
31.38554216867469 [[0.         0.75903614 0.24096386]]


In [343]:
def malicious(  selfPayoffMatrix:np.matrix,
                selfMinimumRequiredPayoff:np.float64,
                opponentPayoffMatrix:np.matrix,
                selfPlayerType:Literal["row", "column"]) -> Tuple[np.float64, np.matrix]:
    A, B = selfPayoffMatrix, opponentPayoffMatrix
    if selfPlayerType.lower() == "row": A, B = A.T, B.T
    U_0 = np.matrix(np.ones_like(A))*selfMinimumRequiredPayoff - A
    k = abs(B.min())+1; U_1 = B+k
    c = -np.matrix(np.ones((A.shape[1])))
    b_0 = np.matrix(np.zeros((U_0.shape[0], 1)))
    b_1 = np.matrix(np.ones((U_1.shape[0], 1)))
    U = np.vstack((U_0, U_1)); b = np.vstack((b_0, b_1))
    bounds = [(0, None)] * U.shape[1]
    result = linprog(c = c, A_ub = U, b_ub = b, bounds = bounds)
    if not result.success: print("Note that the linear programming was not successful")
    d = result.fun
    opponentsPayoff = -d**-1 - k
    maliciousStrategy = np.matrix(result.x).T / -d
    return opponentsPayoff, maliciousStrategy

In [344]:
print(toolbox.Game.fromMatrices(player_1_payoff_matrix = A, player_2_payoff_matrix = B, transposed = True))
print(minimumRequiredValue_row)

            C_0        C_1        C_2
R_0   (16, -41)   (17, 15)  (-31, 83)
R_1  (-45, -19)  (-13, 55)  (22, -43)
R_2   (41, -47)   (18, 67)  (22, -25)
R_3   (-73, 92)   (79, 54)   (66, 57)
-62.7548100349883


In [345]:
opponentsUpperBound, maliciousStrategy = malicious(A, minimumRequiredValue_row, B, "row")
print(round(opponentsUpperBound, 12), maliciousStrategy.T)
print(round(enhancedPayoff_col, 12), enhancedStrategy_col.T)

31.385542168675 [[0.59036145 0.40963855 0.         0.        ]]
31.385542168675 [[0.         0.75903614 0.24096386]]
